In [25]:
import numpy as np
from sympy import Matrix
from random import randint

# Задание 1. Шифр Хилла

Алфавит и его мощность

In [26]:
a = ['я', 'й', 'ц', 'е', 'р', 'о', 'ж', 'д', 'и', 'н', ',']
n = len(a)
n

11

Ценное сообщение из 12 символов

In [27]:
msg = 'яйцерождение'

Три матрицы-ключа

In [28]:
k2 = np.array([[1, 2], [3, 4]])
k3 = np.array([[6, 24, 1], [13, 16, 10], [20, 17, 15]])
k4 = np.array([[4, 3, 2, 1], [7, 6, 5, 8], [12, 11, 10, 9], [13, 16, 15, 14]])

Убедимся в том, что определители матриц не имеют общих делителей с n

In [29]:
d2 = round(np.linalg.det(k2))
d3 = round(np.linalg.det(k3))
d4 = round(np.linalg.det(k4))
print(d2, d3, d4)
if np.gcd(d2, n) == 1 and np.gcd(d3, n) == 1 and np.gcd(d4, n) == 1:
    print('Всё ОК')
else:
    print('Всё плохо!!!')

-2 441 -128
Всё ОК


Зашифруем сообщение с помощью каждого из ключей

In [30]:
def encrypt(msg, k):
    dim = k.shape[0]
    p = []
    enc = ''
    for i in range(len(msg)):
        if i % dim == 0 and i != 0:
            c = k.dot(p) % n
            for j in c:
                enc += a[j]
            p = []
        p.append(a.index(msg[i]))
    c = k.dot(p) % n
    for j in c:
        enc += a[j]
    return enc

enc2, enc3, enc4 = encrypt(msg, k2), encrypt(msg, k3), encrypt(msg, k4)
enc2, enc3, enc4

('цриде,нц,йее', 'реен,оняндян', ',деяжйжйедяр')

Расшифрование работает как надо

In [31]:
def decrypt(enc, k):
    dim = k.shape[0]
    p = []
    dec = ''
    k_inv = np.array(Matrix(k).inv_mod(n))
    for i in range(len(enc)):
        if i % dim == 0 and i != 0:
            c = k_inv.dot(p) % n
            for j in c:
                dec += a[j]
            p = []
        p.append(a.index(enc[i]))
    c = k_inv.dot(p) % n
    for j in c:
        dec += a[j]
    return dec
decrypt(enc2, k2), decrypt(enc3, k3), decrypt(enc4, k4)

('яйцерождение', 'яйцерождение', 'яйцерождение')

Сымитируем вредоносное вмешательство

In [32]:
for _ in range(3):
    enc2 = list(enc2)
    enc2[randint(0, 11)] = a[randint(0, n - 1)]
    enc2 = ''.join(enc2)
    enc3 = list(enc3)
    enc3[randint(0, 11)] = a[randint(0, n - 1)]
    enc3 = ''.join(enc3)
    enc4 = list(enc4)
    enc4[randint(0, 11)] = a[randint(0, n - 1)]
    enc4 = ''.join(enc4)
enc2, enc3, enc4

('цридц,нц,дйе', 'реян,ониндяд', ',дняжйжйедяр')

Расшифруем получившиеся строки

In [33]:
decrypt(enc2, k2), decrypt(enc3, k3), decrypt(enc4, k4)

('яйцежнжднжйя', ',,йеройжцйеж', ',ее,рождение')

# Задание 2. Взлом шифра Хилла

Ключ размер 2х2

In [34]:
k2 = np.array([[1, 3], [3, 1]])

Два сообщения из 12 символов

In [35]:
msg1 = 'рождениеяйце'
msg2 = 'яйцерождение'

Шифруем оба сообщения

In [36]:
enc1, enc2 = encrypt(msg1, k2), encrypt(msg2, k2)
enc1, enc2

('ижоеиджоейян', 'ейянижоеиджо')

Забываем одно из исходных сообщений

In [37]:
msg1 = ''

Восстанавливаем ключ по исходному и зашифрованному сообщению

In [38]:
p, c = [], []
P, C = [], []
for i in range(4):
    p.append(a.index(msg2[i]))
    c.append(a.index(enc2[i]))
    if len(p) == 2:
        P.append(p)
        C.append(c)
        p, c = [], []
P = np.array(P)
C = np.array(C)
k = np.array((Matrix(P).inv_mod(n)  * Matrix(C)) % n, dtype=int)


Расшифровываем забытое сообщение

In [39]:
decrypt(enc1, k)

'рождениеяйце'

# Задание 3. Код Хэмминга

Русский алфавит

In [40]:
a = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н','о', 'п', 'р', 'с', 'т', 'у', 'ф','х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э','ю','я']
n = len(a)
n

32

Присвоим каждой букве пятибитовый двоичный номер

In [41]:
def bin(i):
    r = ''
    while i > 0:
        r += str(i % 2)
        i //= 2
    while len(r) < 5:
        r += '0'
    return r[::-1]

a_bin = [bin(i) for i in range(n)]

Интересное слово из 4 букв

In [42]:
word = 'шифр'

Закодируем его двоичным кодом

In [43]:
enc_bin = ''.join([a_bin[a.index(c)] for c in word])
enc_bin

'11000010001010010000'

Матрицы G и H 

In [44]:
G = np.array([[1, 1, 0, 1], [1, 0, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
H = np.array([[1, 0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, 1]])

Почему матрицы G и H составлены именно так?

Матрица G - матрица-генератор кодов. Преобразует сообщение в виде вектора размерности (4, 1) в вектор размерности (7, 1), добавляя три дополнительных бита четности. Соответственно размер матрицы (7, 4)

Матрица H - проверяет на наличие "испорченных" битов в закодированном сообщении. Возвратит нуль-вектор, если сообщение не было испорчено, иначе укажет испорченный бит. Преобразует вектор размерности (7, 1) в вектор размерности (3, 1). Соответственно размер матрицы (3, 7)

Можно сделать вывод, что Im(G) = Ker(H)

Hamming (7, 4) позволяет найти и исправить 1 испорченных бит в 4-х битовом сообщении (если испорчено 2 бита, то мы можем только узнать об этом, но не исправить). Для этого  добавляются 3 бита четности, которые должны покрывать 4 бита исходного сообщения. Номер бита четности в сообщении будет равен степени двойки (то есть в закодированном сообщении 1ый, 2ой и 4ый биты будут битами четности). Каждый из битов четности будет отвечать за 3 бита данных (значение бита четности будет зависить от значений 3 бит данных, 4 бита вместе должны составлять четное количество единиц).

![Диаграмма](assets/hamming-7-4-ab.jpg)



Закодируем слово с помощью матрицы G

In [45]:
encoded_word = ''
for i in range(0, len(enc_bin), 4):
    p = np.array([int(c) for c in enc_bin[i : i + 4]])
    x = G.dot(p) % 2
    encoded_word += ''.join([str(i) for i in x])
encoded_word

'01111000101010010101000110010000000'

Сымитируем вредоносное вмешательство

In [46]:
broken_words = []
broken1 = list(encoded_word)
for _ in range(1):
    i = randint(0, len(broken1) - 1)
    broken1[i] = '1' if broken1[i] == '0' else '0'
broken1 = ''.join(broken1)
broken_words.append(broken1)

broken2 = list(encoded_word)
for _ in range(2):
    i = randint(0, len(broken2) - 1)
    broken2[i] = '1' if broken2[i] == '0' else '0'
broken2 = ''.join(broken2)
broken_words.append(broken2)

broken3 = list(encoded_word)
for _ in range(3):
    i = randint(0, len(broken3) - 1)
    broken3[i] = '1' if broken3[i] == '0' else '0'
broken3 = ''.join(broken3)
broken_words.append(broken3)

broken4 = list(encoded_word)
for _ in range(4):
    i = randint(0, len(broken4) - 1)
    broken4[i] = '1' if broken4[i] == '0' else '0'
broken4 = ''.join(broken4)
broken_words.append(broken4)

Находим и исправляем ошибки с помощью матрицы H и декодируем сообщение

In [47]:
R = np.array([[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]])
word_bins = []
for broken in broken_words:
    word_bin = ''
    for i in range(0, len(broken), 7):
        r = np.array([int(c) for c in broken[i : i + 7]])
        z = H.dot(r) % 2
        print(z)
        if sum(z) != 0:
            idx = int(''.join([str(n) for n in z[::-1]]), 2) - 1
            r[idx] = 1 if r[idx] == 0 else 0
        p = R.dot(r) % 2
        word_bin += ''.join([str(i) for i in p])
    print('---')
    word_bins.append(word_bin)

[0 0 0]
[0 0 0]
[0 0 0]
[1 1 1]
[0 0 0]
---
[0 0 0]
[1 0 0]
[0 0 0]
[1 0 1]
[0 0 0]
---
[1 1 0]
[0 0 0]
[1 1 1]
[0 1 0]
[0 0 0]
---
[0 1 1]
[0 0 0]
[0 0 0]
[1 0 0]
[0 1 0]
---


Переводим полученный результат в слово из 4-х букв

In [48]:
for word_bin in word_bins:
    result = ''
    for i in range(0, len(word_bin), 5):
        result += a[a_bin.index(word_bin[i : i + 5])]
    print(result)

шифр
шифр
шифр
шифх


# Задание 4. Код Хэмминга?

Данную задачу можно решить с помощью кода Хэмминга.

Пронумеруем все клетки доски слева направо сверху вниз от 0 до 63. 1, 2, 4, 8, 16, 32 клетки будут битами четности, всего 6 бит четности. Матрица H должна получить на вход вектор из 64 бит и вернуть вектор из 6 бит, который будет двоичным представлением номера клетки с ключом. Размерность матрицы H (6, 64).

Матрица будет заполняться так же, как и в случае hamming (7, 4)

![Как заполнить матрицу](assets/bGc9m.jpg)